### Step 1 – Load the Dataset


In [2]:
import pandas as pd
area_fire= pd.read_csv(r'working data\CA (cleaned+unit id) wildfire area data (1984 to 2023) .csv')
#weather_fire= pd.read_csv(r'CA_Weather (cleaned up) 1984-2023.csv')

Cleaning up the data, dropping extra column


In [3]:
area_fire.head()

#check the number of features and amount of entries
print('no. of data, no. of features:', area_fire.shape)
#check the value and type of each column(features)
print('data info:', area_fire.info())

area_fire= area_fire.dropna()


no. of data, no. of features: (22261, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22261 entries, 0 to 22260
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id            11200 non-null  float64
 1   ALARM_DATE     10618 non-null  object 
 2   CONT_DATE      8737 non-null   object 
 3   UNIT_ID        22195 non-null  object 
 4   GIS_ACRES      11200 non-null  float64
 5   Shape__Area    11200 non-null  float64
 6   Shape__Length  11200 non-null  float64
 7   YEAR_          11123 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1.4+ MB
data info: None


tabula: convert pdf table into pandas df (failed coz it became a weird list, multiple values in one rows?)
tried using online pdf table to excel and import as xlsx

import numpy as np
#print(np.where(city['MACS-ID']== unitid)) -- not work due to differnt length

# Find values in unitid but not in city['MACS-ID']
missing_unitids = list(set(unitid) - set(city['MACS-ID']))

print(missing_unitids)

In [4]:
#all the station mentioned in area_fire
unitid = list(area_fire["UNIT_ID"].unique())

agentcity= pd.read_excel(r'agency+unit id.xlsx',sheet_name='Table 5')


#converting the mentioned fire station code to name
agent_name= city_mentioned['Display Value']
agent_name #all the mentioned fire station NAME



NameError: name 'city_mentioned' is not defined

ca fire_agency-list is from 
https://firescope.caloes.ca.gov/Governance/Group%20Documents/MACS-ID-Agency-List.pdf

In [ ]:
##testing out if the fire station mentioned matches '"ca fire-Agency-List.xlsx" since it has city information
cafireagencywithcity= pd.read_excel(r'ca fire-Agency-List.xlsx')
cafireagencywithcity=cafireagencywithcity.dropna()

relevantmasc_id= cafireagencywithcity[cafireagencywithcity['MACS-ID'].isin(unitid)]
city_withfire= relevantmasc_id[['MACS-ID', 'CITY']]
city_withfire.head()
#city_withfire is the conversion between unitid in fire to city name
# Create a mapping dictionary (Stored Value → Display Value)
city_mapping = dict(zip(city_withfire['MACS-ID'], city_withfire['CITY']))

# Map values to create the 'city_name' column in area_fire
area_fire["city_name"] = area_fire['UNIT_ID'].map(city_mapping)
#dropped missing values
area_fire= area_fire.dropna()

#the list of city we are gonna use to match with the weather stations
citymatchtoweather= relevantmasc_id['CITY']
citymatchtoweather


8               Camino
24             Arcadia
54      San Bernardino
55      San Bernardino
61            Monterey
             ...      
932            Visalia
953            Vallejo
955          Camarillo
967               Weed
1004          Yosemite
Name: CITY, Length: 89, dtype: object

for weather data: we have to align the weather station to the fire stations 
with their corresponding city


In [ ]:
allweather_stn = pd.read_excel(r'ghcnd-stations-CA only.xlsx')
allweather_stn = allweather_stn.drop(columns=['GSN FLAG', 'HCN/CRN FLAG'])
allweather_stn.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,WMO ID
0,USC00040006,39.0333,-122.4333,601.1,CA,ABBOTT MINE,
1,USC00040010,38.2178,-121.2014,26.5,CA,ACAMPO 5NE,
2,USR0000CACT,34.4458,-118.2000,792.5,CA,ACTON CALIFORNIA,
3,USC00040014,34.4939,-118.2714,863.5,CA,ACTON ESCONDIDO FC261,
4,USC00040024,34.5833,-117.4167,869.0,CA,ADELANTO,


In [ ]:
#matching the list of city to the weather stations to get their weather station id

weather_stnreq = allweather_stn[allweather_stn['NAME'].apply(lambda x: any(city.lower() in x.lower() for city in citymatchtoweather))]
weather_stnreq

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,WMO ID
39,USC00040161,41.4903,-120.5439,1332.9,CA,ALTURAS,
40,USC00040162,41.5147,-120.3642,1617.6,CA,ALTURAS 9 ENE,
41,USW00094299,41.4836,-120.5614,1335.9,CA,ALTURAS MUNI AP,
84,US1CALA0021,34.1617,-118.0285,198.7,CA,ARCADIA 2.1 NNE,
98,USW00024283,40.9783,-124.1047,64.3,CA,ARCATA EUREKA AP,
...,...,...,...,...,...,...,...
3122,USW00053150,37.7592,-119.8208,2017.8,CA,YOSEMITE VILLAGE 12 W,
3124,USC00049866,41.7025,-122.6417,826.6,CA,YREKA,
3125,US1CASK0005,41.7326,-122.6495,820.5,CA,YREKA 0.9 WNW,
3126,US1CASK0002,41.6651,-122.6208,895.2,CA,YREKA 4.5 S,


In [ ]:
#list of weather station id we need to get from ftp
weatherid_list = weather_stnreq["ID   "]
weatherid_list

39      USC00040161
40      USC00040162
41      USW00094299
84      US1CALA0021
98      USW00024283
           ...     
3122    USW00053150
3124    USC00049866
3125    US1CASK0005
3126    US1CASK0002
3127    US1CASK0030
Name: ID   , Length: 413, dtype: object

In [ ]:
import os
import requests

# Base URL
base_url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_station/"

# List of station IDs (modify as needed)


# Directory to save downloaded files
download_dir = "weather_data"
os.makedirs(download_dir, exist_ok=True)

for station_id in weatherid_list:
    file_name = f"{station_id}.csv.gz"  # NOAA files are in .csv.gz format
    file_url = base_url + file_name
    local_file_path = os.path.join(download_dir, file_name)

    # Download the file
    response = requests.get(file_url, stream=True)
    
    if response.status_code == 200:
        with open(local_file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name} (Status Code: {response.status_code})")


Now: make a column in area_fire, indicating that these observations are Fire=True

This is to put all 413 weather csv into 1 dataframe

In [ ]:
import os
import gzip
import pandas as pd

# Define the path where your datasets are stored
data_folder = "weather_data"

# Define the path where you want to save the converted files
converted_folder = "converted_files"

# Create the converted folder if it doesn't exist
os.makedirs(converted_folder, exist_ok=True)

# Get a list of all .csv.gz files and then convert them into .csv
all_files = [f for f in os.listdir(data_folder) if f.endswith('csv.gz')]

for file in all_files:
    # Open the .gz file in text mode and read using pandas
    with gzip.open(f"{data_folder}/{file}", 'rt', encoding='utf-8') as f:  # open in text mode with UTF-8 encoding
        try:
            # Read the CSV file, handle bad lines, and ensure proper separator
            df = pd.read_csv(f, low_memory=False, on_bad_lines='skip', sep=',')
            
            # Print row count for debugging (before saving)
            print(f"Read {file} with {len(df)} rows.")

            # Save the converted .csv file in the new folder (without index)
            converted_file_path = os.path.join(converted_folder, f"{file.split('.')[0]}.csv")
            df.to_csv(converted_file_path, index=False)
            
            print(f"Saved converted file to: {converted_file_path}")
            
        except Exception as e:
            print(f"Error processing {file}: {e}")




Read US1CAAL0007.csv.gz with 5014 rows.
Saved converted file to: converted_files\US1CAAL0007.csv
Read US1CAAL0022.csv.gz with 1383 rows.
Saved converted file to: converted_files\US1CAAL0022.csv
Read US1CAAL0024.csv.gz with 473 rows.
Saved converted file to: converted_files\US1CAAL0024.csv
Read US1CAAL0031.csv.gz with 605 rows.
Saved converted file to: converted_files\US1CAAL0031.csv
Read US1CAAM0004.csv.gz with 936 rows.
Saved converted file to: converted_files\US1CAAM0004.csv
Read US1CAAM0014.csv.gz with 469 rows.
Saved converted file to: converted_files\US1CAAM0014.csv
Read US1CAAM0016.csv.gz with 2172 rows.
Saved converted file to: converted_files\US1CAAM0016.csv
Read US1CABT0002.csv.gz with 5677 rows.
Saved converted file to: converted_files\US1CABT0002.csv
Read US1CABT0003.csv.gz with 198 rows.
Saved converted file to: converted_files\US1CABT0003.csv
Read US1CABT0005.csv.gz with 8556 rows.
Saved converted file to: converted_files\US1CABT0005.csv
Read US1CABT0007.csv.gz with 2861 r

In [ ]:
import pandas as pd
#test to see if it works
testing_csv = pd.read_csv(r'converted_files\US1CAAL0007.csv')

In [ ]:
'''
with open(r'', 'rb') as file:
    content = file.read(50)  # Read the first 20 bytes
    print(content)
'''
import chardet

with open(r'converted_files\US1CAAL0007.csv', 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    print(result)


{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [5]:
df = pd.read_csv(r'converted_files\US1CAAL0007.csv', encoding='latin1')
#it worked, use this encoding after this

In [12]:
import os
import pandas as pd
import glob

# Path to the folder where your CSV files are stored
folder_path = "converted_files"

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Define the columns you want to use as the header (change as needed)
default_columns = ['Source.Name', 'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7']

# Specify the chunk size (e.g., 1 million rows at a time)
chunk_size = 1000000

# Initialize an empty list to store individual DataFrames
chunks = []

# Loop through all CSV files and add headers if missing, then read in chunks
for file in csv_files:
    try:
        # Read the first few rows to check if the file has a header
        df_preview = pd.read_csv(file, encoding='ISO-8859-1', nrows=5, header=None)

        # Check if the first row contains invalid values that suggest it's not a header
        if df_preview.iloc[0].isna().all() or df_preview.iloc[0].apply(lambda x: isinstance(x, (int, float))).all():
            print(f"Adding header to {file}")
            
            # If no valid header, we will write the default header to the CSV
            with open(file, 'r+', encoding='ISO-8859-1') as f:
                content = f.readlines()
                content[0] = ",".join(default_columns) + '\n'  # Modify the first line to add header
                f.seek(0)
                f.writelines(content)  # Rewrite the file with the header added
        else:
            print(f"File {file} already has a header.")
        
        # Read the CSV file in chunks after adding header (or confirming it's already there)
        for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):
            # Drop columns 5-8 (indexes 4-7)
            chunk = chunk.drop(chunk.columns[4:8], axis=1)
            chunks.append(chunk)
    
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Concatenate all chunks into a single DataFrame
combined_df = pd.concat(chunks, ignore_index=True)

# Optionally, save the combined DataFrame to a new CSV
combined_df.to_csv("combined_output.csv", index=False)

# Print the first few rows to confirm
print(combined_df.head())


File converted_files\US1CAAL0007.csv already has a header.
File converted_files\US1CAAL0022.csv already has a header.
File converted_files\US1CAAL0024.csv already has a header.
File converted_files\US1CAAL0031.csv already has a header.
File converted_files\US1CAAM0004.csv already has a header.
File converted_files\US1CAAM0014.csv already has a header.
File converted_files\US1CAAM0016.csv already has a header.
File converted_files\US1CABT0002.csv already has a header.
File converted_files\US1CABT0003.csv already has a header.
File converted_files\US1CABT0005.csv already has a header.
File converted_files\US1CABT0007.csv already has a header.
File converted_files\US1CABT0008.csv already has a header.
File converted_files\US1CABT0009.csv already has a header.
File converted_files\US1CABT0010.csv already has a header.
File converted_files\US1CABT0011.csv already has a header.
File converted_files\US1CABT0012.csv already has a header.
File converted_files\US1CABT0013.csv already has a heade

C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):
C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):


File converted_files\USC00040385.csv already has a header.
File converted_files\USC00040439.csv already has a header.
File converted_files\USC00040444.csv already has a header.
File converted_files\USC00040519.csv already has a header.
File converted_files\USC00040521.csv already has a header.
File converted_files\USC00040819.csv already has a header.
File converted_files\USC00040823.csv already has a header.
File converted_files\USC00040824.csv already has a header.
File converted_files\USC00041444.csv already has a header.
File converted_files\USC00041715.csv already has a header.
File converted_files\USC00041758.csv already has a header.


C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):
C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):


File converted_files\USC00042147.csv already has a header.
File converted_files\USC00042148.csv already has a header.
File converted_files\USC00042150.csv already has a header.
File converted_files\USC00042319.csv already has a header.


...

C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):


File converted_files\USW00053150.csv already has a header.
File converted_files\USW00053152.csv already has a header.
File converted_files\USW00093110.csv already has a header.
File converted_files\USW00093121.csv already has a header.
File converted_files\USW00093124.csv already has a header.
File converted_files\USW00093134.csv already has a header.


C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):
C:\Users\natal\AppData\Local\Temp\ipykernel_24072\1603579778.py:40: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, encoding='ISO-8859-1', chunksize=chunk_size, on_bad_lines='skip'):


File converted_files\USW00093193.csv already has a header.
File converted_files\USW00093206.csv already has a header.
File converted_files\USW00093210.csv already has a header.
File converted_files\USW00093243.csv already has a header.
File converted_files\USW00094299.csv already has a header.


MemoryError: Unable to allocate 112. MiB for an array with shape (1, 14687396) and data type float64

In [ ]:
#this is done on the original area_fire, if you load it again, these dropping code would be needed to clean up area_fire
#area_fire=area_fire.drop(['CONT_DATE'],axis=1)
#area_fire=area_fire.drop(['_id'],axis=1)



In [ ]:

# Join and drop unmatched values
df = df.join(scores, how='inner')  # 'inner' keeps only matching rows

print(df)

NameError: name 'df' is not defined

In [ ]:
area_date=area_fire.iloc[:, 0:4]
area_date.set_index(['ALARM_DATE'])
weather_fire.set_index(['DATE'])


,FIRE_START_DAY,PRECIPITATION,MAX_TEMP,MIN_TEMP,AVG_WIND_SPEED,TEMP_RANGE,WIND_TEMP_RATIO,LAGGED_PRECIPITATION,LAGGED_AVG_WIND_SPEED,SEASON,DAY_OF_YEAR
DATE,,,,,,,,,,,
31/12/2023,False,0.00,60,51,4.92,9,0.082000,0.62,5.558571,Winter,365
30/12/2023,False,0.62,62,54,8.50,8,0.137097,0.62,5.558571,Winter,364
29/12/2023,False,0.00,65,52,3.80,13,0.058462,0.00,5.175714,Winter,363
28/12/2023,False,0.00,64,50,5.59,14,0.087344,0.50,5.782857,Winter,362
27/12/2023,False,0.00,62,52,5.59,10,0.090161,1.83,6.198571,Winter,361
...,...,...,...,...,...,...,...,...,...,...,...
5/1/1984,False,0.00,74,49,5.14,25,0.069459,0.00,5.100000,Winter,5
4/1/1984,False,0.00,76,45,4.70,31,0.061842,0.00,5.090000,Winter,4
3/1/1984,False,0.00,70,47,5.37,23,0.076714,0.00,5.220000,Winter,3


### Step 2 – Preprocess the Dataset
 
 The features are all on different scales. To bring them all to a common scale, we’ll use the **StandardScaler** that transforms the features to have zero mean and unit variance:

##### Thoughts
run a PCA to see which top 2-3 factors are the most crucial?

In [ ]:
import sklearn 
from sklearn.preprocessing import StandardScaler
#separate features and
features = ['sepal length', 'sepal width', 'petal length', 'petal width']
std_scaler = StandardScaler()
scaled_cafire = std_scaler.fit_transform(ca_fire)

NameError: name 'ca_fire' is not defined

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split